# Advanced Querying Techniques in Elasticsearch for BiorXiv Data Analysis

Elasticsearch, renowned for its speed and scalability, is an indispensable tool for data scientists and researchers working with large datasets like those from BiorXiv. BiorXiv provides a rich corpus of preprint publications in the life sciences, offering a wealth of data for analysis. In this guide, we delve into sophisticated querying techniques to extract meaningful insights from BiorXiv data using Elasticsearch. We'll explore everything from basic keyword searches to complex aggregations and filters.

### Establishing a Secure Connection to Elasticsearch


Before diving into queries, establishing a secure connection to your Elasticsearch cluster is paramount. This ensures that your data interactions are encrypted and protected. Here’s a brief refresher on setting up a secure connection:

In [9]:
%%time

import ssl

# Path to the CA certificate
ca_cert_path = '/workspace/repos/osl/rxiv-restapi/containers/esconfig/certs/http_ca.crt'
# Create an SSL context
ssl_context = ssl.create_default_context(cafile=ca_cert_path)
# Create a connection to Elasticsearch with authentication and SSL context
es = Elasticsearch(
    ["https://es:9200"],
    basic_auth=("elastic", "worksfine"),
    ssl_context=ssl_context
)

CPU times: user 5.07 ms, sys: 199 µs, total: 5.27 ms
Wall time: 4.04 ms


### Index JSON Data into Elasticsearch
**Before we can search, you must have the data indexed in Elasticsearch.** 
**Here's a simplified function to read the JSON file and index its contents.** 
**This example assumes that your JSON data is an array of objects, each representing a document to be indexed.**

In [8]:
import json
from elasticsearch import Elasticsearch

def index_json_data(es: Elasticsearch, file_path: str, index_name: str) -> None:
    """
    Reads data from a JSON file and indexes it into Elasticsearch.

    Parameters:
    es (Elasticsearch): An Elasticsearch client instance.
    file_path (str): The path to the JSON file.
    index_name (str): The name of the Elasticsearch index where data will be stored.
    """
    # Load JSON data from the file
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)

    # Assuming `data` is a list of documents
    for doc in data:
        # Index each document
        res = es.index(index=index_name, document=doc)
        # print(res['result'])


### Indexing Data into Elasticsearch

In [10]:
%%time

# Assuming index_json_data is a previously defined function that indexes data from a JSON file to Elasticsearch
# Path to the JSON file containing data to be indexed
file_path = '/workspace/repos/osl/rxiv-restapi/docs/notebooks/data/biorxiv_2022-01-01_2024-01-11.json'
# Name of the Elasticsearch index
index_name = 'biorxiv'
# Index data from the specified JSON file into Elasticsearch
index_json_data(es, file_path, index_name)

CPU times: user 3min 28s, sys: 18 s, total: 3min 46s
Wall time: 27min 26s


## Querying Analyses for BiorXiv Data


### Create a Search Function

**After indexing the data, we can create a function to perform searches using the Elasticsearch client.**

In [13]:
def search_data(es: Elasticsearch, index_name: str, query: dict) -> list:
    """
    Performs a search query in an Elasticsearch index and returns only the document data.
    search_data function is structured to perform a search query and return document hits directly,

    Parameters:
    es (Elasticsearch): An Elasticsearch client instance.
    index_name (str): The name of the Elasticsearch index to search in.
    query (dict): The search query in Elasticsearch Query DSL format.

    Returns:
    list: A list of documents from the search results, each represented as a dictionary.
    """
    try:
        response = es.search(index=index_name, query=query)
        # Extract only the '_source' from each hit
        documents = [hit['_source'] for hit in response['hits']['hits']]
        return documents
    except Exception as e:
        print(f"Search failed: {e}")
        return []

### Keyword Searches: The Basics
#### Search for documents with a specific title.


These query examples illustrate the flexibility of Elasticsearch's Query DSL to retrieve specific data based on various search criteria.

Creating a variety of Elasticsearch query combinations involves using different aspects of the Elasticsearch Query DSL (Domain Specific Language) to retrieve specific documents based on your criteria. Below are several examples of query combinations that can be used to retrieve keys and values from the JSON data provided

Each of these queries can be passed to the `search_data` function you've defined to retrieve documents from Elasticsearch based on the specified criteria. Remember to replace `index_name` with the name of your index when calling the function:

### Match Query


Keyword searches are the foundation of data retrieval in Elasticsearch. For instance, finding all BiorXiv papers related to CRISPR:


In [21]:
%%time 

query = {
    "match": {
        "abstract": "CRISPR"
    }
}

results = search_data(es, index_name, query)

print("Results: ",len(results), "\n")
# Display the results directly
for result in results:
    # Assuming you want to display each document's key-value pairs
    print(result)

print("-" * 25)

Results:  10 

{'doi': '10.1101/2024.01.05.574328', 'title': 'CRISPR-repressed toxin-antitoxin provides population-level immunity against diverse anti-CRISPR elements', 'authors': 'Li, M.; Shu, X.; Wang, R.; Li, Z.; Xue, Q.; Liu, C.; Cheng, F.; Zhao, H.; Wang, J.; Liu, J.; Hu, C.; Li, J.; Ouyang, S.', 'author_corresponding': 'Ming Li', 'author_corresponding_institution': 'Institute of Microbiology, CAS', 'date': '2024-01-05', 'version': '1', 'license': 'cc_no', 'category': 'Microbiology', 'jatsxml': 'https://www.biorxiv.org/content/early/2024/01/05/2024.01.05.574328.source.xml', 'abstract': 'Prokaryotic CRISPR-Cas systems are highly vulnerable to phage-encoded anti-CRISPR (Acr) factors. How CRISPR-Cas systems protect themselves remains unclear. Here, we uncovered a broad-spectrum anti-anti-CRISPR strategy involving a phage-derived toxic protein. Transcription of this toxin is normally reppressed by the CRISPR-Cas effector, but is activated to halt cell division when the effector is inh

### Term Query
Retrieve documents where the `license` field exactly matches the specified value.


In [62]:
%%time

query = {
    "term": {
        "license": {
            "value": "cc_by_nc_nd"
        }
    }
}

# Execute the search
results = search_data(es, index_name, query)

print("Results: ",len(results), "\n")
# Display the results directly
for result in results:
    # Assuming you want to display each document's key-value pairs
    print(result)

print("-" * 25)

Results:  10 

{'doi': '10.1101/2022.01.10.475621', 'title': 'Ecosystem size-induced environmental fluctuations affect the temporal dynamics of community assembly mechanisms', 'authors': 'Bier, R. L.; Vass, M.; Szekely, A. J.; Langenheder, S.', 'author_corresponding': 'Raven L Bier', 'author_corresponding_institution': 'Department of Ecology and Genetics/Limnology, Uppsala University, Sweden', 'date': '2022-01-11', 'version': '1', 'license': 'cc_by_nc_nd', 'category': 'Ecology', 'jatsxml': 'https://www.biorxiv.org/content/early/2022/01/11/2022.01.10.475621.source.xml', 'abstract': 'Understanding processes that determine community membership and abundance is important for many fields from theoretical community ecology to conservation. However, spatial community studies are often conducted only at a single timepoint despite the known influence of temporal variability on community assembly processes. Here we used a spatiotemporal study to determine how environmental fluctuation difference

### Range Query
Find documents published within a specific date range.


In [63]:
%%time

query = {
    "range": {
        "date": {
            "gte": "2022-12-01",
            "lte": "2022-12-31"
        }
    }
}

# Execute the search
results = search_data(es, index_name, query)

print("Results: ",len(results), "\n")
# Display the results directly
for result in results:
    # Assuming you want to display each document's key-value pairs
    print(result)

print("-" * 25)

Results:  10 

{'doi': '10.1101/2022.01.12.475145', 'title': 'Exercise reprograms the inflammatory landscape of multiple stem cell compartments during mammalian aging', 'authors': 'Liu, L.; Kim, S.; Buckley, M. T.; Reyes, J. M.; Kang, J.; Tian, L.; Wang, T.; Lieu, A.; Mao, M.; Mateo-Rodriguez, C.; Ishak, H.; Jeong, M.; Wu, J. C.; Goodell, M. A.; Brunet, A.; Rando, T. A.', 'author_corresponding': 'Thomas A Rando', 'author_corresponding_institution': 'Stanford University', 'date': '2022-12-25', 'version': '2', 'license': 'cc_no', 'category': 'Cell Biology', 'jatsxml': 'https://www.biorxiv.org/content/early/2022/12/25/2022.01.12.475145.source.xml', 'abstract': 'Exercise has the ability to rejuvenate stem cells and improve tissue homeostasis and regeneration in aging animals. However, the cellular and molecular changes elicited by exercise have not been systematically studied across a broad range of cell types in stem cell compartments. To gain better insight into the mechanisms by which e


### Bool Query
Combine multiple search criteria. For example, search for documents by authors in a specific category and with a specific license.


In [64]:
%%time

query = {
    "bool": {
        "must": [
            {"match": {"authors": "Colman-Lerner"}},
            {"match": {"category": "Systems Biology"}},
            {"term": {"license": "cc_by_nc_nd"}}
        ]
    }
}

# Execute the search
results = search_data(es, index_name, query)

print("Results: ",len(results), "\n")
# Display the results directly
for result in results:
    # Assuming you want to display each document's key-value pairs
    print(result)

print("-" * 25)

Results:  10 

{'doi': '10.1101/2022.10.06.511167', 'title': 'High selectivity of frequency induced transcriptional responses', 'authors': 'Givre, A.; Colman-Lerner, A.; Ponce-Dawson, S.', 'author_corresponding': 'Silvina Ponce-Dawson', 'author_corresponding_institution': 'School of Natural and Exact Sciences, University of Buenos Aires', 'date': '2022-10-07', 'version': '1', 'license': 'cc_by_nc_nd', 'category': 'Systems Biology', 'jatsxml': 'https://www.biorxiv.org/content/early/2022/10/07/2022.10.06.511167.source.xml', 'abstract': 'Cells continuously interact with their environment, detect its changes and generate responses accordingly. This requires interpreting the variations and, in many occasions, producing changes in gene expression. In this paper we use information theory and a simple transcription model to analyze the extent to which the resulting gene expression is able to identify and assess the intensity of extracellular stimuli when they are encoded in the amplitude, dura

### Match phrase Query
Use a match_phrase to search for documents with titles that contain specific patterns.


In [65]:
%%time

query = {
    "match_phrase": {
        "title": "Carm1 regulates the speed of"
    }
}

# Execute the search
results = search_data(es, index_name, query)

print("Results: ",len(results), "\n")
# Display the results directly
for result in results:
    # Assuming you want to display each document's key-value pairs
    print(result)

print("-" * 25)

Results:  6 

{'doi': '10.1101/2022.10.03.510647', 'title': 'Carm1 regulates the speed of C/EBPa-induced transdifferentiation by a cofactor stealing mechanism', 'authors': 'Garcia, G. T.; Kowenz-Leutz, E.; Tian, T. V.; Klonizakis, A.; Lerner, J.; De Andres-Aguayo, L.; Berenguer, C.; Carmona, M. P.; Casadesus, M. V.; Bulteau, R.; Francesconi, M.; Leutz, A.; Zaret, K. S.; Zaret, K. S.; Peiro, S.', 'author_corresponding': 'Achim Leutz', 'author_corresponding_institution': 'MDC, Berlin', 'date': '2022-10-04', 'version': '1', 'license': 'cc_by_nc_nd', 'category': 'Cell Biology', 'jatsxml': 'https://www.biorxiv.org/content/early/2022/10/04/2022.10.03.510647.source.xml', 'abstract': 'Cell fate decisions are driven by lineage-restricted transcription factors but how they are regulated is incompletely understood. The C/EBP-induced B cell to macrophage transdifferentiation (BMT) is a powerful system to address this question. Here we describe that C/EBP with a single arginine mutation (C/EBPR35A)


### Multi-Match Query
Search for a text across multiple fields.


In [66]:
%%time

query = {
    "multi_match": {
        "query": "transcription",
        "fields": ["title", "abstract"]
    }
}

# Execute the search
results = search_data(es, index_name, query)

print("Results: ",len(results), "\n")
# Display the results directly
for result in results:
    # Assuming you want to display each document's key-value pairs
    print(result)

print("-" * 25)

Results:  10 

{'doi': '10.1101/2021.04.02.438179', 'title': 'Mechanism of transcription modulation by the transcription-repair coupling factor', 'authors': 'Paudel, B. P.; Xu, Z.-Q.; Jergic, S.; Oakley, A. J.; Sharma, N.; Brown, S. H.; Bouwer, J. C.; Lewis, P.; Dixon, N. E.; Van Oijen, A. M.; Ghodke, H.', 'author_corresponding': 'Harshad  Ghodke', 'author_corresponding_institution': 'University of Wollongong', 'date': '2022-02-22', 'version': '3', 'license': 'cc_no', 'category': 'Biophysics', 'jatsxml': 'https://www.biorxiv.org/content/early/2022/02/22/2021.04.02.438179.source.xml', 'abstract': 'Elongation by RNA polymerase is dynamically modulated by accessory factors. The transcription-repair coupling factor (TRCF) recognizes distressed RNAPs and either rescues transcription or initiates transcription termination. Precisely how TRCFs choose to execute either outcome remains unclear. With Escherichia coli as a model, we used single-molecule assays to study dynamic modulation of elong

### Advanced Filtering: Beyond Keywords

Filtering allows for more refined searches, such as retrieving documents within a specific date range or by particular authors, enhancing the precision of your data analysis.


In [71]:
%%time

query = {
    "bool": {
        "must": {
            "match": {"title": "Molecular Biology"}
        },
        "filter": {
            "range": {
                # publish_date
                "date": {
                    "gte": "2022-01-01",
                    "lte": "2023-12-31"
                }
            }
        }
    }
}

results = search_data(es, index_name, query)

print("Results: ",len(results), "\n")
# Display the results directly
for result in results:
    # Assuming you want to display each document's key-value pairs
    print(result)

print("-" * 25)

Results:  10 

{'doi': '10.1101/2023.05.24.542151', 'title': 'POMBOX: a fission yeast toolkit for molecular and synthetic biology', 'authors': 'Hebra, T.; Smrckova, H.; Elkatmis, B.; Prevorovsky, M.; Pluskal, T.', 'author_corresponding': 'Tomas Pluskal', 'author_corresponding_institution': 'Institute of Organic Chemistry and Biochemistry of the Czech Academy of Sciences, Praha, Czech Republic', 'date': '2023-05-24', 'version': '1', 'license': 'cc_by_nc_nd', 'category': 'Synthetic Biology', 'jatsxml': 'https://www.biorxiv.org/content/early/2023/05/24/2023.05.24.542151.source.xml', 'abstract': 'Schizosaccharomyces pombe is a popular model organism in molecular biology and cell physiology. With its ease of genetic manipulation and growth, supported by in-depth functional annotation in the PomBase database and genome-wide metabolic models, S. pombe is an attractive option for synthetic biology applications. However, S. pombe currently lacks modular tools for generating genetic circuits wit

### Aggregation Query


Aggregations are pivotal for summarizing data, enabling the analysis of trends across thousands of documents. For example, an aggregation query to count publications by category:

In [80]:
query = {
    "size": 0,  # Important for aggregations, as you might not be interested in the hits but the aggregation results
    "aggs": {
        "categories": {  # This name must match what you access in the results
            "terms": {
                "field": "category.keyword",  # Adjust based on your actual field name
                "size": 10  # Adjust the size as needed
            }
        }
    }
}


In [82]:
def search_data_with_aggregation(es: Elasticsearch, index_name: str, query: Dict[str, Any]) -> List[Dict[str, Any]]:
    try:
        response = es.search(index=index_name, body=query)
        if 'aggregations' in response:
            # Safely access the 'categories' aggregation results
            categories_agg = response.get('aggregations', {}).get('categories', {}).get('buckets', [])
            return categories_agg  # Directly return the categories aggregation results
        else:
            print("No aggregations found in the response.")
            return []
    except Exception as e:
        print(f"Search failed: {e}")
        return []

#### Aggregate data, such as counting documents by category.

Checks if the response contains an 'aggregations' key and processes the results accordingly. For aggregation queries, it returns the aggregation results directly. 

In [83]:
%%time

# Assuming 'es', 'index_name', and 'query' are properly defined
results = search_data_with_aggregation(es, index_name, query)

# Display the aggregation results
if results:
    for category in results:
        print(f"Category: {category['key']}, Count: {category['doc_count']}")
else:
    print("No results found.")

Category: Neuroscience, Count: 40174
Category: Microbiology, Count: 18624
Category: Bioinformatics, Count: 18114
Category: Cell Biology, Count: 12558
Category: Biophysics, Count: 10406
Category: Evolutionary Biology, Count: 10384
Category: Biochemistry, Count: 8914
Category: Immunology, Count: 8834
Category: Cancer Biology, Count: 8824
Category: Ecology, Count: 8802
CPU times: user 2.54 ms, sys: 266 µs, total: 2.8 ms
Wall time: 96 ms


---

### The biorxiv database was downloaded from the medrxivr library

The json file used is structure as a list of dictionaries downloaded by MedrdR from a specific date range from 2022 to 2024 and contain 000 papers of data

### Final Note: The Impact of Advanced Querying on Research


Advanced querying techniques in Elasticsearch empower researchers to navigate and analyze the vast repository of BiorXiv data with unprecedented depth and precision. From basic keyword searches to sophisticated aggregations and scripted calculations, Elasticsearch facilitates a comprehensive understanding of the life sciences landscape. By harnessing these querying capabilities, researchers can accelerate discovery, foster innovation, and contribute to the advancement of science.



---